<a href="https://colab.research.google.com/github/LM1997610/Data-Mining/blob/main/HW_4_DMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# DMT 2023 - Homework 4

## Group composition:

<br>

------------YOUR TEXT STARTS HERE------------

<br>

- Mazzucco, Luca, 1997610\
- Zilviano, Paolo, 1916518

<br>

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

## Homework 4
The homework consists of two parts:

  1. Text Representation

  and

  2. Deep Learning

> Ensure that the notebook can be faithfully reproduced by anyone (hint: pseudo random number generation).

> If you need to set a random seed, set it to `709`.

> If multiple code cells are provided for a single part, it is **NOT** mandatory to use them all.

# Part 1
In this part of the homework, you have to deal with Text Representation.

Import **ALL** the Python packages that you need for Part 1.

In [ ]:
%%capture

#REMOVE_OUTPUT#
!pip install --upgrade --no-cache-dir gdown
#YOUR CODE STARTS HERE#

!pip install -U sentence-transformers
!pip install gensim
from gensim.models import Word2Vec

import time; import json; import string
import numpy as np; import pandas as pd
from tqdm.auto import tqdm; from tabulate import tabulate

!pip install py3langid
from py3langid.langid import LanguageIdentifier, MODEL_FILE

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

## Part 1.1
The company **F**antastic **S**olution sells products. Customers can leave product reviews on their platform. The company wants to classify the reviews into positive and negative.

Their requirements are unclear: they mention both accuracy and calculation time, but it is not known which is more important to them. :'(

They also forbid you to do a hyper-parameter optimisation.
(why? :O )

To help you (?), they have already pre-processed the data. They have translated each text into a random language.

The best thing to do is to provide them with a list of models that can best meet their (unclear) requirements.

### 1.1.1
Download the data from the Drive link (code already provided).

In [ ]:
#REMOVE_OUTPUT#
!gdown 1X6QnCcOgnNEBQ1xnilmPWqDIs7bRrQof

Downloading...
From: https://drive.google.com/uc?id=1X6QnCcOgnNEBQ1xnilmPWqDIs7bRrQof
To: /home/ec2-user/SageMaker/Data-Mining/FS_reviews.jsonl
100%|██████████████████████████████████████| 18.3M/18.3M [00:00<00:00, 63.1MB/s]


### 1.1.2
Understand (!) and pre-process *(general term!)* the data.
Divide the data according to your needs.

> No specific request


In [ ]:
#YOUR CODE STARTS HERE#

# building the dataframe:
with open('FS_reviews.jsonl', 'r') as json_file:
    json_list = list(json_file)

lista = [json.loads(review) for review in tqdm(json_list)]

df = pd.DataFrame.from_dict(lista)

tqdm.pandas()
print("data.shape:", df.shape)
print("\ndata.columns:", list(df.columns))


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

  0%|          | 0/20210 [00:00<?, ?it/s]

data.shape: (20210, 10)

data.columns: ['unique_id', 'product_name', 'product_type', 'helpful', 'rating', 'title', 'date', 'review_text', 'reviewer', 'reviewer_location']


In [ ]:
#YOUR CODE STARTS HERE#

# data cleaning:

df.drop_duplicates(keep="first", inplace=True)

df.replace('', None, inplace=True);
df = df[df['review_text'] != "ERROR"]
df = df[df['title'] != "ERROR"]

df.reset_index(drop=True, inplace=True)
#df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')
df["rating"] = df["rating"].astype(float).astype(int)

print("\n data.shape:", df.shape)

df["real_id"] = df["unique_id"].str.split(":").str[0]
df["product_name"]  = df["product_name"].str.split(":").str[0]

df = df.drop('unique_id', axis=1)

columns = ['real_id', 'date', 'title', 'review_text',
           'rating', 'helpful', 'reviewer', 'reviewer_location',
           'product_name', 'product_type']

df = df[columns]

print()
print(tabulate(df["rating"].value_counts().to_frame(), headers=["rating", "count"],
                                                  numalign='center', stralign="center"))


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#


 data.shape: (18550, 10)

 rating    count
--------  -------
   5       11110
   4       3784
   1       2332
   2       1324


In [ ]:
#YOUR CODE STARTS HERE#

#################################################
def tokenization(text):

    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))

    new_text = text.translate(table).replace("\n", " ")
    new_text = " ".join(new_text.lower().split())

    return new_text
#####################################################

#pre-processing text:

df["processed_text"] = df["review_text"].progress_apply(lambda desc: tokenization(desc))

df["class"] = [0 if i < 3 else 1 for i in df['rating']] # binary classification

# language detection:
identifier = LanguageIdentifier.from_pickled_model(MODEL_FILE, norm_probs=True)
identifier.set_languages(['en', 'it', 'pl', "ko", "es", "tl", "sw", "ms", "de"])

df["lingua"] = df["processed_text"].progress_apply(lambda desc: identifier.classify(desc)[0])

print()
print(tabulate(df["lingua"].value_counts().to_frame(), headers=["lingua", "count"],
                                                   numalign='center', stralign="center"))


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

  0%|          | 0/18550 [00:00<?, ?it/s]

  0%|          | 0/18550 [00:00<?, ?it/s]


 lingua    count
--------  -------
   en      9629
   it      1838
   pl      1623
   ko      1563
   es      1532
   de      1397
   tl       539
   sw       389
   ms       40


Provide an explanation for your preprocessing *(general term!)*.

**Use at most 3 sentences.**

Duplicate entries have been eliminated, and mis-registered values have been discarted.\
Regarding the text  we executed removal of punctuation, extra-whitespace, and line breaks.\
Finally, some counts to understand the dataset imbalance regarding ratings and languages.

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.3
Choose at least 1 and a maximum of 3 encodings. Encode the data.

> P.S. If you need it, Word2Vec has a version for Documents

In [ ]:
#YOUR CODE STARTS HERE#

# 1 Model -> Sentence Embedding using Sentence Transformers:
first_model = 'distiluse-base-multilingual-cased-v1'

model_one = SentenceTransformer(first_model)

embed_one = model_one.encode(df["processed_text"], show_progress_bar=True)

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Batches:   0%|          | 0/580 [00:00<?, ?it/s]

In [ ]:
#YOUR CODE STARTS HERE#

# 2 Model -> Sentence Embedding using Sentence Transformers:
second_model = 'paraphrase-multilingual-MiniLM-L12-v2'

model_two = SentenceTransformer(second_model)

embed_two = model_two.encode(df["processed_text"], show_progress_bar=True)

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Batches:   0%|          | 0/580 [00:00<?, ?it/s]

In [ ]:
#YOUR CODE STARTS HERE#

X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df["class"],
                                                    test_size=0.2,
                                                    random_state=709)
# 3 Model -> Word embedding using Word2Vec:

sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Give a reason for your choices.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

Selected two pretrained-multilingual models from SentenceTransformer, trained on data for 50+ languages, this adequately meets our needs.
Then a model based on word embeddings provided by Word2Vec was chosen due to the inability to implement a successful classifier using suggested Doc2Vec...

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.4
Choose **ONE** classifier for **EACH** encoding. Train the classifiers.

In [ ]:
#YOUR CODE STARTS HERE#

from sklearn.linear_model import LogisticRegression;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.svm import SVC;

classifiers = [SVC(random_state=709), KNeighborsClassifier(), LogisticRegression(max_iter=1000)]

######################################

def train_the_classifiers(model, tr_x, tr_y):

    start = time.time()
    model.fit(tr_x, tr_y)
    duration = time.time() - start

    return round(duration, 4)
#######################################################

embeddings = embed_one

train_x_a, test_x_a, train_y_a, test_y_a = train_test_split(embeddings, df["class"],
                                                    test_size=0.2,
                                                    shuffle=True, random_state = 709)


d_0 = {"Classifiers": str(classifiers[0]).split("(")[0],
       "FIT_Time": train_the_classifiers(classifiers[0], train_x_a, train_y_a)}


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#

embeddings = embed_two

train_x_b, test_x_b, train_y_b, test_y_b= train_test_split(embeddings, df["class"],
                                                    test_size=0.2,
                                                    shuffle=True, random_state = 709)


d_1 = {"Classifiers": str(classifiers[1]).split("(")[0],
       "FIT_Time": train_the_classifiers(classifiers[1], train_x_b, train_y_b)}



#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#


output = train_the_classifiers(classifiers[2], X_train, y_train)

d_2 = {"Classifiers": str(classifiers[2]).split("(")[0],
       "FIT_Time": output}


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Provide a justification for your choices.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

Distinct classifiers have been chosen for each encoding, selecting the most traditional algorithms available

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.5
Obtain the metrics you want to show the company.

In [ ]:
#YOUR CODE STARTS HERE#

#########################################################

def get_the_metrics(model, tst_x, tst_y, old_diz):

    t1 =   time.time()
    predicted = model.predict(tst_x)
    old_diz["Pred_time"] = round(time.time() - t1, 4)

    old_diz["Accuracy"] = round(accuracy_score(tst_y, predicted), 4)

    return old_diz

#########################################################


d_0 = get_the_metrics(classifiers[0], test_x_a, test_y_a, d_0)


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#

d_1 = get_the_metrics(classifiers[1], test_x_b, test_y_b, d_1)

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#

d_2 = get_the_metrics(classifiers[2], X_test, y_test, d_2)


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Provide a rationale for the choice of metrics.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

Two metrics have been chosen to express "calculation time" - specifically, the training time and prediction time.\
Additionally we took into account the accuracy demanded by the company


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.6
Provide the company with all the information it needs to choose the pipeline it prefers.

In [ ]:
#YOUR CODE STARTS HERE#

metrics_df = pd.DataFrame.from_dict([d_0, d_1, d_2])

name_1 = "-".join(first_model.split("-")[2:])
name_2 = "-".join(second_model.split("-")[2:])

metrics_df.index = [name_1, name_2, "Word2vec"]

print()
print(tabulate(metrics_df, headers=metrics_df.columns, numalign='center', stralign="center"))
print()

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#


                           Classifiers        FIT_Time    Pred_time    Accuracy
---------------------  --------------------  ----------  -----------  ----------
multilingual-cased-v1          SVC            41.0111      10.1812      0.8876
    MiniLM-L12-v2      KNeighborsClassifier    0.0033      0.6728       0.8604
      Word2vec          LogisticRegression     0.6737      0.0016       0.8113



Describe what you have supplied and why.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

Here is a table displaying characteristics and the desired performance for our three models.


Also, explain how the company could decide between the classifiers you have produced.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

In cases where comparable accuracy scores are achieved, it is advisable to opt for the simpler and faster model.\
Then define a balance between fit_time and prediction_time which will depend on the specific needs of the company\
SVC is known to be computationally heavier, but in this case it exhibits the best performance.

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

# Part 2
In this part of the homework, you have to deal with Deep Learning.

Import here **ALL** the Python packages that you need for Part 2.

In [ ]:
%%capture

#REMOVE_OUTPUT#
#YOUR CODE STARTS HERE#

!pip install torchtext

import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

from torchtext.data.functional import to_map_style_dataset


#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Part 2.1
You have to use the same data as in Part 1, but you can use whatever adjustments you have made to it (only Part 1.1.2).

### 2.1.1
Prepare the data structures you will need.

In [ ]:
#YOUR CODE STARTS HERE#

tuple_list = list(zip(df["rating"], df["processed_text"]))

train_size = int(0.8 * len(df))
test_size = len(df) - train_size

torch.manual_seed(709)

train_d, test_d = random_split(tuple_list, [train_size, test_size])

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(iter(train_d)), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

print("\nesempio:", vocab(['here', 'is', 'an', 'example']))

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1


all_classes = set([label for (label, text) in train_d])
all_classes.add(3)
print("\nClasses", all_classes, end = " ")

num_class = len(all_classes)
print("Number classes:", num_class, "\n")

#YOUR CODE ENDS HERE#
#THIS IS LINE 40#


esempio: [564, 7, 104, 5030]

Classes {1, 2, 3, 4, 5} Number classes: 5 



In [ ]:
#YOUR CODE STARTS HERE#

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("\n Device:", device, torch.cuda.get_device_name(0), "on GPU =", torch.cuda.is_available())

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)



#YOUR CODE ENDS HERE#
#THIS IS LINE 40#


 Device: cuda Tesla T4 on GPU = True


Briefly explain relevant parts of your code.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

Organizing the data in list of tuples, splitting them into training and testing datasets, setting up tokenization and preprocessing functions, initialize a vocabulary, and obtaining the number of classes in the data.\
Then the "collate_batch" function to processes a collected batch of data by applying pipelines for label processing and text processing, it converts the text into a tensor, and it returns the label tensor, processed text tensor, and offsets.


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.2
Define your model

In [ ]:
#YOUR CODE STARTS HERE#


class TextClassificationModel(torch.nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()

        #Computes sums or means of ‘bags’ of embeddings, without instantiating the intermediate embeddings.
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)

        self.fc = torch.nn.Linear(embed_dim, num_class)
        #self.layer1 = torch.nn.Linear(embed_dim, num_class)

        #we can use multiple linear layers in sequence --> Deep Learning!

        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)

        return self.fc(embedded)



vocab_size = len(vocab)
emb_size = 84
model = TextClassificationModel(vocab_size, emb_size, num_class).to(device)


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#



#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

Briefly explain relevant parts of your code.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

The code defines a TextClassificationModel class from torch.nn.Module.\
It initializes the model with an embedding layer (EmbeddingBag) and a linear layer (Linear).\
The forward method performs the forward pass of the model, taking input text and offsets, embedding the text, and passing it through the linear layer.\
The model is then instantiated with the defined vocabulary size, embedding size, and number of classes.

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.3
Train and optimize your model

In [ ]:
#YOUR CODE STARTS HERE#

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

        #print(  idx, log_interval, idx % log_interval )
        if idx % log_interval == 0 and idx>0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader), total_acc/total_count))

            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

    return total_acc/total_count



#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#

# Hyperparameters
EPOCHS = 10 # epoch
LR = 7   # learning rate
BATCH_SIZE = 6   # batch size for training

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=LR)
optimizer = torch.optim.Adadelta(model.parameters(), lr=LR)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter, test_iter = random_split(tuple_list, [train_size, test_size])

train_dataset = to_map_style_dataset((train_iter)) #Convert iterable-style dataset to map-style dataset.
test_dataset = to_map_style_dataset((train_iter)) #Convert iterable-style dataset to map-style dataset.

num_train = int(len(train_dataset) * 0.75)


split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)

valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, collate_fn=collate_batch)


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

In [ ]:
#YOUR CODE STARTS HERE#

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

| epoch   1 |   500/ 1855 batches | accuracy    0.589
| epoch   1 |  1000/ 1855 batches | accuracy    0.605
| epoch   1 |  1500/ 1855 batches | accuracy    0.615
-----------------------------------------------------------
| end of epoch   1 | time:  9.95s | valid accuracy    0.600 
-----------------------------------------------------------
| epoch   2 |   500/ 1855 batches | accuracy    0.640
| epoch   2 |  1000/ 1855 batches | accuracy    0.648
| epoch   2 |  1500/ 1855 batches | accuracy    0.671
-----------------------------------------------------------
| end of epoch   2 | time:  9.97s | valid accuracy    0.621 
-----------------------------------------------------------
| epoch   3 |   500/ 1855 batches | accuracy    0.695
| epoch   3 |  1000/ 1855 batches | accuracy    0.716
| epoch   3 |  1500/ 1855 batches | accuracy    0.735
-----------------------------------------------------------
| end of epoch   3 | time:  9.91s | valid accuracy    0.634 
-------------------------------

Briefly explain relevant parts of your code.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------


The train function trains the model by iterating over batches, calculating loss, performing backpropagation, and updating the model's parameters.\
The evaluate function evaluates the model's performance on a given set.\
Then defines hyperparameters, creates optimizer and scheduler objects, and iterates over epochs, training the model and evaluating on the validation dataset.

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.4
Show the performance of your model

In [ ]:
#YOUR CODE STARTS HERE#


print('\nChecking the results of test dataset...')
accu_test = evaluate(test_dataloader)
print(' ... test accuracy: {:8.3f}'.format(accu_test))



#YOUR CODE ENDS HERE#
#THIS IS LINE 40#


Checking the results of test dataset...
 ... test accuracy:    0.847


In [ ]:
#YOUR CODE STARTS HERE#


def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

news_label = {1: "really_bad", 2: "bad", 3: "mid", 4 : "good", 5 : "really_good"}

ex_text_str = "The SuperGadget X5 Multi-Tool is a total disappointment.\
                    Poor build quality, ineffective tools, \
                    and an unappealing design make it a regrettable purchase.\
                    Save your money and explore better options."

model = model.to("cpu")

print("This is a %s review" %news_label[predict(ex_text_str, text_pipeline)])


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

This is a really_bad review


In [ ]:
#YOUR CODE STARTS HERE#


text_test = "The XYZ Portable Bluetooth Speaker exceeded my expectations in every way.\
            The sound quality is exceptional, delivering crystal-clear audio with rich bass.\
            The battery life is impressive, providing hours of uninterrupted music playback.\
            Additionally, the build quality is top-notch, showcasing a sleek and sturdy design.\
            I highly recommend this speaker to anyone looking for a premium audio experience on the go."

print("This is a %s review" %news_label[predict(text_test, text_pipeline)], "\n")


#YOUR CODE ENDS HERE#
#THIS IS LINE 40#

This is a really_good review 



Briefly explain relevant parts of your code.

**Use at most 3 sentences.**

[comment]: <> (#SHOW_CELL#)

----------YOUR TEXT STARTS HERE----------

The predict function uses the pre-trained model to predict the sentiment label (ranging from 1 to 5) of a given text.\
Then we apply this function to two example reviewes and prints the corresponding sentiment label for testing purposes\

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

## Part 2.2

### 2.2.1
How would a Deep Learning model (of the kind we have seen) behave in the case where a word was never seen during training?
Answer on both practical and theoretical aspects.

**Use at most 3 sentences.**

----------YOUR TEXT STARTS HERE----------

<br>

Deep learning model use word embeddings, such as Word2Vec or SentenceEmbedding, so when it encounters a word that was never seen during training, the model attempt to find the nearest neighbors in the embedding space for the unseen word.
Neural LanguageModels can use similarity among words
embeddings to generalize and predict unseen words.


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>


# Submission instructions

Follow the instructions precisely
1. Run the whole notebook up to this point
  * Click on this text cell and go to Runtime > Run before
  * It is essential that the notebook can be run from start to finish without us having to change anything.
2. Download the current notebook
  * File -> Download -> Download .ipynb
3. Upload the downloaded notebook in the current runtime
  * In the left sidebar menu, open the last tab (Files)
  * Click on the first button under the File title (Page with arrow pointing up)
  * In the new window, upload the notebook just downloaded



4. To turn the notebook into a pdf you need to install a Tex extension: run the next cell of code:

In [ ]:
#REMOVE_CELL#
!apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/sh: apt-get: command not found


5. Now, run the next cell to set some cell tags in the notebook file.

In [ ]:
#REMOVE_CELL#
import nbformat as nbf
from glob import glob
notebooks = glob("/content/**/*.ipynb", recursive=True)

text_search_dict = ["YOUR TEXT STARTS HERE","#YOUR CODE STARTS HERE#"]

for ipath in notebooks:
    print(ipath)
    ntbk = nbf.read(ipath, nbf.NO_CONVERT)

    for cell in ntbk.cells:
        cell_type = cell['cell_type'] == "code"

        cell_tags = cell.get('metadata', {}).get('tags', [])
        cell_tags = set(cell_tags)

        if cell_type == 0: #markdown
          if (cell['source'][0]!="#" and text_search_dict[cell_type] not in cell['source'] and "pagebreak" not in cell['source']) or "#REMOVE_CELL#" in cell['source']:
            cell_tags.add("remove_cell")

        else: #code
          #if text_search_dict[cell_type] not in cell['source'] or "#REMOVE_CELL#" in cell['source']:
          if "#REMOVE_CELL#" in cell['source']:
            cell_tags.add("remove_cell")
          elif "#REMOVE_OUTPUT#" in cell['source']:
            cell_tags.add("remove_output")

        if len(cell_tags) > 0:
            cell['metadata']['tags'] = list(cell_tags)

    nbf.write(ntbk, ipath)

6. Now, run the next cell to transform the notebook to PDF without the code cells.
  * The file will appear in the menu on the left (refresh if needed). The file should be named `DMT2023_HW4.pdf`
  * Rename this file **Surname1_Surname2_DMT2023_HW4_report.pdf** (e.g. Becchetti_Siciliano_DMT2023_HW4_report.pdf). Sort the surnames alphabetically.
  * Click on it and using the right button or the three dots at its side, click on download to transfer it to your local PC
  

In [ ]:
#REMOVE_CELL#
!jupyter nbconvert /content/DMT2023_HW4.ipynb --to pdf --TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' -TagRemovePreprocessor.remove_all_outputs_tags='{"remove_output"}'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[NbConvertApp] WARNING | pattern '/content/DMT2023_HW4.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    S

7. Now, rename the downloaded notebook **Surname1_Surname2_DMT2023_HW4_notebook.ipynb** (e.g. Becchetti_Siciliano_DMT2023_HW4_notebook.ipynb). Sort the surnames alphabetically.

8. Now that you have both files, you need to upload them to Classroom. Only the student with the surname that comes first in alphabetical order **must** upload the files. Only the files uploaded by this student will be graded.